Resources:
- https://quantum-computing.ibm.com/composer/files/new
- https://wybiral.github.io/quantum/
- https://quantumcomputing.stackexchange.com/questions/29454/why-do-quantum-computing-simulators-have-the-measurement-function

In [74]:
import numpy as np
import scipy as sp

In [75]:
norm_matr = lambda state: state / sp.linalg.norm(state)

class QState:
    def __init__(self, qbit_cnt, info=False):
        self.qbit_cnt = qbit_cnt
        # create state |00...0> with qbit_cnt amount of zeros
        self.state = np.zeros(2**self.qbit_cnt, dtype=complex)
        self.state[0] = 1

        if info:
            max_bin_width = len(bin(len(self.state) - 1)[2:])
            binary_str = bin(0)[2:].zfill(max_bin_width)
            print("Initial state: |{}>".format(binary_str))

    # aplly op_matrix to single qubit at pos (first qubit -> pos=0)
    def sing_qbit_op(self, op_matrix, pos):
        # suppose we habe 4 qubits, and want to apply the X gate
        # on the thrid qubit, aka at pos=2
        # notice that (I ⊗ I ⊗ X ⊗ I)|0000> can be rewritten
        # as I|0> ⊗ I|0> ⊗ X|0> ⊗ I|0>, thus only applying the
        # gate at pos=2 -> final state is |0010>

        # further notice that previously I=I_2, where I_m is the
        # identity matrix of size mxm, now notice I_2 ⊗ I_2 = I_4
        # and I_2 ⊗ I_2 ⊗ I_2 = I_8, so generally I^{⊗n}=I_{2^n}

        # putting all together, we can see that applying a gate at pos=n,
        # there will be n identity matrices infront and qbit_cnt-n-1 after
        # so we can finally write the gate to apply on the whole state as:
        # I_{2^pos} ⊗ op_matrix ⊗ I_{2^{qbit_cnt-pos-1}}
        idmats_left = np.eye(2**pos, dtype=complex)
        idmats_right = np.eye(2**(self.qbit_cnt-pos-1))

        gate = np.kron(np.kron(idmats_left, op_matrix), idmats_right)
        self.state = np.matmul(gate, self.state)
    
    # show all states including the impossible one (unless
    # otherwise noted) and show according probabilities
    def show_state_and_probs(self, reduced=False):
        max_len = 0
        for element in self.state:
            if reduced and element == 0+0j:
                continue

            curr_len = max(len(str(element.real)), len(str(element.imag)))
            if curr_len > max_len:
                max_len = curr_len
        
        max_bin_width = len(bin(len(self.state) - 1)[2:])

        for i, element in enumerate(self.state):
            if reduced and element == 0+0j:
                continue

            real_part_str = "{:.{width}f}".format(element.real, width=max_len)
            imaginary_part_str = "{:.{width}f}".format(element.imag, width=max_len)
            formatted_element = "{}+{}i".format(real_part_str, imaginary_part_str)

            binary_str = bin(i)[2:].zfill(max_bin_width)

            prob_str = str(round((np.absolute(element)**2)*100, 4))

            if i > 0:
                print("+ {}|{}>\t-> {}%".format(formatted_element, binary_str, prob_str))
            else:    
                print("  {}|{}>\t-> {}%".format(formatted_element, binary_str, prob_str))
        

    ###
    ### SINGLE-QUBIT GATES ###
    ###

    def pauli_x(self, pos):
        op_matrix = np.array([
            [0,1],
            [1,0]
        ])
        self.sing_qbit_op(op_matrix, pos)

    def pauli_y(self, pos):
        op_matrix = np.array([
            [0,0-1j],
            [0+1j,0]
        ])
        self.sing_qbit_op(op_matrix, pos)

    def pauli_z(self, pos):
        op_matrix = np.array([
            [1,0],
            [0,-1]
        ])
        self.sing_qbit_op(op_matrix, pos)

    def hadamard(self, pos):
        op_matrix = norm_matr(np.array([
            [1,1],
            [1,-1]
        ]))
        self.sing_qbit_op(op_matrix, pos)

    ###
    ### MULTI-QUBIT GATES ###
    ###
    
    # strictly only if control bit above target bit
    def cnot(self, ctrl_pos, targ_pos):
        p_0 = np.array([
            [1,0],
            [0,0]
        ])
        p_1 = np.array([
            [0,0],
            [0,1]
        ])
        pauli_x = np.array([
            [0,1],
            [1,0]
        ])

        idmats_before = np.eye(2**ctrl_pos, dtype=complex)
        idmats_after = np.eye(2**(self.qbit_cnt-ctrl_pos-1), dtype=complex)
        ctrl_0_case = np.kron(np.kron(idmats_before, p_0), idmats_after)

        idmats_before = np.eye(2**ctrl_pos, dtype=complex)
        idmats_between = np.eye(2**(targ_pos-ctrl_pos-1), dtype=complex)
        idmats_after = np.eye(2**(self.qbit_cnt-targ_pos-1), dtype=complex)
        ctrl_1_case = np.kron(np.kron(np.kron(np.kron(idmats_before, p_1), idmats_between), pauli_x), idmats_after)

        self.state = np.matmul(ctrl_0_case + ctrl_1_case, self.state)






In [80]:
### Testing

qs = QState(4, True)
qs.pauli_x(1)
print("Mid state vector: ", qs.state)
qs.show_state_and_probs(True)
qs.cnot(1, 3)

print("Final state vector: ", qs.state)
qs.show_state_and_probs(True)

Initial state: |0000>
Mid state vector:  [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
+ 1.000+0.000i|0100>	-> 100.0%
Final state vector:  [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
+ 1.000+0.000i|0101>	-> 100.0%
